### 豆瓣评分的预测

在这个项目中，我们要预测一部电影的评分，这个问题实际上就是一个分类问题。给定的输入为一段文本，输出为具体的评分。 在这个项目中，我们需要做：
- 文本的预处理，如停用词的过滤，低频词的过滤，特殊符号的过滤等
- 文本转化成向量，将使用三种方式，分别为tf-idf, word2vec以及BERT向量。 
- 训练逻辑回归和朴素贝叶斯模型，并做交叉验证
- 评估模型的准确率


In [1]:
#导入数据处理的基础包
import numpy as np
import pandas as pd

#导入用于计数的包
from collections import Counter

#导入tf-idf相关的包
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer

#导入模型评估的包
from sklearn import metrics

#导入与word2vec相关的包
from gensim.models import KeyedVectors

#导入与bert embedding相关的包，关于mxnet包下载的注意事项参考实验手册
from bert_embedding import BertEmbedding
import mxnet

#包tqdm是用来对可迭代对象执行时生成一个进度条用以监视程序运行过程
from tqdm import tqdm

#导入其他一些功能包
import requests
import os


### 1. 读取数据并做文本的处理
需要完成以下几步操作：
- 去掉无用的字符如！&，可自行定义
- 中文分词
- 去掉低频词

In [2]:
#读取数据
data = pd.read_csv('data/DMSC.csv')
#观察数据格式
data.head()

,ID,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,10,Avengers Age of Ultron,复仇者联盟2,2017-01-22,11,影志,2015-04-30,4,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,381
2,20,Avengers Age of Ultron,复仇者联盟2,2017-01-22,21,随时流感,2015-04-28,2,奥创弱爆了弱爆了弱爆了啊！！！！！！,120
3,30,Avengers Age of Ultron,复仇者联盟2,2017-01-22,31,乌鸦火堂,2015-05-08,4,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,30
4,40,Avengers Age of Ultron,复仇者联盟2,2017-01-22,41,办公室甜心,2015-05-10,5,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,16


In [3]:
#只保留数据中我们需要的两列：Comment列和Star列
data = data[['Comment','Star']]
#观察新的数据的格式
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国。,3
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,4
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,2
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,4
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,5


In [4]:
# 这里的star代表具体的评分。但在这个项目中，我们要预测的是正面还是负面。我们把评分为1和2的看作是负面，把评分为3，4，5的作为正面
data['Star']=(data.Star/3).astype(int)
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国。,1
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1


#### 任务1： 去掉一些无用的字符

In [5]:
tqdm.pandas(desc='douban') #查看处理进度
data['Comment']=data['Comment'].progress_apply(lambda x:x.replace('，','').replace('。',''))

douban: 100%|██████████████████████████████████████████████████████████████| 212506/212506 [00:00<00:00, 562982.34it/s]


In [6]:
# 观察新的数据的格式
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国,1
1,“一个没有黑暗面的人不值得信任” 第二部剥去冗长的铺垫开场即高潮、一直到结束会有人觉得只剩...,1
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战...,1
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹...,1


#### 任务2：使用结巴分词对文本做分词

In [7]:
import jieba
def comment_cut(content):
    return list(jieba.cut(content.strip()))

# 输出进度条
tqdm.pandas(desc='apply')
data['comment_processed'] = data['Comment'].progress_apply(comment_cut)

apply:   0%|                                                                                | 0/212506 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\chxue\AppData\Local\Temp\jieba.cache
Loading model cost 0.858 seconds.
Prefix dict has been built successfully.
apply: 100%|█████████████████████████████████████████████████████████████████| 212506/212506 [00:53<00:00, 3994.68it/s]


In [10]:
data.head()

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,"[连, 奥创, 都, 知道, 整容, 要, 去, 韩国]"
1,“一个没有黑暗面的人不值得信任” 第二部剥去冗长的铺垫开场即高潮、一直到结束会有人觉得只剩...,1,"[“, 一个, 没有, 黑暗面, 的, 人, 不, 值得, 信任, ”, , 第二部, 剥..."
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0,"[奥创, 弱, 爆, 了, 弱, 爆, 了, 弱, 爆, 了, 啊, ！, ！, ！, ！,..."
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战...,1,"[与, 第一集, 不同, 承上启下, 阴郁, 严肃, 但, 也, 不会, 不, 好看, 啊,..."
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹...,1,"[看毕, 我, 激动, 地, 对, 友人, 说, 等等, 奥创, 要, 来, 毁灭, 台北,..."


#### 任务3：设定停用词并去掉停用词

In [12]:
# 读取下载的停用词表，并保存在列表中
with open("data/stopWord.json","r",encoding='utf-8') as f:
    stopWords = f.read().split("\n")

# 去除停用词
def rm_stop_word(wordList):
    return [word for word in wordList if word not in stopWords]

#这行代码中.progress_apply()函数的作用等同于.apply()函数的作用，只是写成.progress_apply()函数才能被tqdm包监控从而输出进度条。
data['comment_processed'] = data['comment_processed'].progress_apply(rm_stop_word)

apply: 100%|█████| 212506/212506 [00:35<00:00, 5936.44it/s]


In [15]:
data.head()

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,"[奥创, 知道, 整容, 韩国]"
1,“一个没有黑暗面的人不值得信任” 第二部剥去冗长的铺垫开场即高潮、一直到结束会有人觉得只剩...,1,"[一个, 没有, 黑暗面, 值得, 信任, , 第二部, 剥去, 冗长, 铺垫, 开场, ..."
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0,"[奥创, 弱, 爆, 弱, 爆, 弱, 爆]"
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战...,1,"[第一集, 不同, 承上启下, 阴郁, 严肃, 不会, 好看, 本来, 喜欢, 漫威, 电影..."
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹...,1,"[看毕, 激动, 友人, 说, 奥创, 毁灭, 台北, 厚, 拍了拍, 肩膀, 没事, 反正..."


#### 任务4：去掉低频词，出现次数少于10次的词去掉

In [17]:
#确实得建立大词典来统计所有词词频
word_dic={}
for comment in data['comment_processed']:
    for word in comment:
        if word in word_dic:
            word_dic[word]+=1
        else:
            word_dic[word]=1
print(len(word_dic))

106235


In [19]:
print(list(word_dic.items())[:10])

[('奥创', 302), ('知道', 6022), ('整容', 79), ('韩国', 1633), ('一个', 13852), ('没有', 18448), ('黑暗面', 31), ('值得', 3056), ('信任', 223), (' ', 98052)]


In [20]:
def rm_low_freq_word(wordList):
    return [w for w in wordList if word_dic[w]>=10]

data['comment_processed'] = data['comment_processed'].progress_apply(rm_low_freq_word)

apply: 100%|███| 212506/212506 [00:00<00:00, 269018.58it/s]


In [21]:
data.head()

,Comment,Star,comment_processed
0,连奥创都知道整容要去韩国,1,"[奥创, 知道, 整容, 韩国]"
1,“一个没有黑暗面的人不值得信任” 第二部剥去冗长的铺垫开场即高潮、一直到结束会有人觉得只剩...,1,"[一个, 没有, 黑暗面, 值得, 信任, , 第二部, 冗长, 铺垫, 开场, 高潮, ..."
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0,"[奥创, 弱, 爆, 弱, 爆, 弱, 爆]"
3,与第一集不同承上启下阴郁严肃但也不会不好看啊除非本来就不喜欢漫威电影场面更加宏大单打与团战...,1,"[第一集, 不同, 承上启下, 阴郁, 严肃, 不会, 好看, 本来, 喜欢, 漫威, 电影..."
4,看毕我激动地对友人说等等奥创要来毁灭台北怎么办厚她拍了拍我肩膀没事反正你买了两份旅行保险惹...,1,"[激动, 友人, 说, 奥创, 毁灭, 台北, 厚, 肩膀, 没事, 反正, 买, 两份, ..."


### 2. 把文本分为训练集和测试集
选择语料库中的20%作为测试数据，剩下的作为训练数据

In [23]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size = 0.2)
x_train = train['comment_processed']
x_test = test['comment_processed']
y_train = train['Star']
y_test = test['Star']

### 3. 把文本转换成向量的形式

在这个部分我们会采用三种不同的方式:
- 使用tf-idf向量
- 使用word2vec
- 使用bert向量

转换成向量之后，我们接着做模型的训练

#### 任务6：把文本转换成tf-idf向量

In [24]:
comments_train_concat = x_train.apply(lambda x:' '.join(x))
comments_test_concat = x_test.apply(lambda x:' '.join(x))

In [27]:
x_train.head()

188015    [不停, 穿越, 不停, 转换,  , 确实, 冲击,  , 跟随, 剧情,  , 完整, ...
193033                                                 [星半]
163310    [ , 其实,  , 没有, 纸醉金迷, ......, 林萧, 有点, 失望, .......]
101549                       [演员, 喜欢, 没, 演技, 撑, 不起, 一部, 电影]
43776                                               [太, 难看]
Name: comment_processed, dtype: object

In [26]:
comments_train_concat.head()

188015    不停 穿越 不停 转换   确实 冲击   跟随 剧情   完整 紧凑 没 说   恍惚 ~ ~
193033                                                  星半
163310                其实   没有 纸醉金迷 ...... 林萧 有点 失望 .......
101549                               演员 喜欢 没 演技 撑 不起 一部 电影
43776                                                 太 难看
Name: comment_processed, dtype: object

In [28]:
vectorizer = CountVectorizer()  #便于稀疏矩阵存储更有效率
trans = TfidfTransformer()  #根据“词 词 词”生成句向量

word_count_train = vectorizer.fit_transform(comments_train_concat) #fit表示既训练又转化
tfidf_train = trans.fit_transform(word_count_train)

word_count_test = vectorizer.transform(comments_test_concat) #fit表示既训练又转化
tfidf_test = trans.transform(word_count_test)

print(tfidf_train.shape, tfidf_test.shape)

(170004, 14880) (42502, 14880)


In [29]:
tfidf_train

<170004x14880 sparse matrix of type '<class 'numpy.float64'>'
	with 1468830 stored elements in Compressed Sparse Row format>

#### 任务7：把文本转换成word2vec向量

In [30]:
# 使用KeyedVectors.load_word2vec_format()函数加载预训练好的词向量文件
model = KeyedVectors.load_word2vec_format('data/sgns.zhihu.word')

In [33]:
model['今天'].shape[0]

300

In [49]:
# 由于数据量大需要长时间训练，这里就用小样本试一下效果
x_train = x_train[:80]
x_test = x_test[:20]
y_train = y_train[:80]
y_test = y_test[:20]
len(x_train)

80

In [50]:
#利用词向量取平均构造句向量
sentence_len=model['今天'].shape[0]
def get_sentence_vec(comment):
    sentence_vec=np.zeros(sentence_len)
    word_num=0
    for word in comment:
        if word in model:
            sentence_vec+=model[word]
            word_num+=1
    return sentence_vec/word_num

word2vec_train = np.vstack(x_train.progress_apply(get_sentence_vec))
word2vec_test = np.vstack(x_test.progress_apply(get_sentence_vec))

print (word2vec_train.shape, word2vec_test.shape)


apply: 100%|██████████████| 80/80 [00:00<00:00, 215.14it/s]

apply: 100%|██████████████| 20/20 [00:00<00:00, 392.58it/s]

(80, 300) (20, 300)


In [51]:
# 查看向量，发现存在空值
print(word2vec_train[:10])
print(np.isnan(word2vec_train).any())

[[-0.085519    0.43515207 -0.17389666 ...  0.20313986  0.073142
  -0.07703887]
 [        nan         nan         nan ...         nan         nan
          nan]
 [-0.00253413  0.31285712  0.03232525 ...  0.06110513  0.09487825
  -0.0657875 ]
 ...
 [-0.090169    0.42740925 -0.217206   ... -0.065442    0.09200576
  -0.041123  ]
 [-0.04754096  0.42249026 -0.143586   ...  0.05339443  0.03275817
  -0.080588  ]
 [-0.16413033  0.3194075  -0.00936734 ...  0.084864    0.14636583
  -0.0699775 ]]
True


In [52]:
# 去掉空值
word2vec_train[np.isnan(word2vec_train)]=0
word2vec_test[np.isnan(word2vec_test)] = 0

In [53]:
# 检查后无空值
print(np.isnan(word2vec_train).any())
print(np.isnan(word2vec_test).any())

False
False


#### 任务8：把文本转换成bert向量

In [8]:
embedding_en = BertEmbedding()

In [9]:
ctx = mxnet.cpu()
embedding = BertEmbedding(ctx=ctx, dataset_name='wiki_multilingual')

D:\Downloads\Anaconda\envs\NLP_class\lib\site-packages\gluonnlp\model\bert.py:693: UserWarning: wiki_cn/wiki_multilingual will be deprecated. Please use wiki_cn_cased/wiki_multilingual_uncased instead.
  warnings.warn('wiki_cn/wiki_multilingual will be deprecated.'


In [10]:
result = embedding(['今天'])

In [11]:
result #查看数据结构，发现result[0][1][0]为“今天”的向量

[(['今', '天'],
  [array([ 1.26106322e-01,  2.66802341e-01,  8.72310027e-02, -6.03290349e-02,
          -7.75884330e-01, -1.13786548e-01,  3.59727591e-01,  1.85813323e-01,
          -1.47403002e+00, -1.19097352e-01, -7.38840699e-02, -7.87430704e-01,
          -9.21352804e-02,  1.61335051e-01, -8.64775032e-02,  2.56176621e-01,
           8.67899284e-02, -1.32034823e-01, -8.10163245e-02, -1.74523041e-01,
           3.95636708e-02,  7.05541968e-02, -1.02931105e-01, -2.12510914e-01,
           7.06533492e-01, -6.93008751e-02,  8.60075727e-02, -2.62604266e-01,
          -1.59369946e+00, -1.49633080e-01, -1.63492024e-01,  5.32592773e-01,
          -4.69610304e-01, -6.60765246e-02, -1.39505550e-01, -3.14024031e-01,
           7.26260841e-02,  1.34167528e+00, -4.96392064e-02, -3.96719128e-01,
           6.35222569e-02, -1.24420702e-01, -6.83844239e-02,  9.81433690e-03,
          -3.70132998e-02, -3.15945178e-01,  6.09729528e-01,  1.38048172e-01,
          -6.59693405e-02, -9.41009894e-02, -7.492

In [91]:
# 将comment数据用bert模型向量化
vec_len = result[0][1][0].shape[0]

def process_word(w):
    vec_com = np.zeros(vec_len)
    num = len(w)
    res = embedding([w])
    k = len(res[0][0])
    for i in range(k):
        vec_com += res[0][1][i]
    return vec_com/k

# 利用词向量平均法求句向量，输入词典类型
def comm_vec(c):
    vec_com = np.zeros(vec_len)
    coun = 0
    for w in c:
        if w in model:
            vec_com += process_word(w)
            coun += 1
    return vec_com/coun



In [94]:
bert_train = np.vstack(x_train.progress_apply(comm_vec))
bert_test = np.vstack(x_test.progress_apply(comm_vec))


apply: 100%|███████████████| 80/80 [05:19<00:00,  4.00s/it]

apply: 100%|███████████████| 20/20 [01:29<00:00,  4.48s/it]


### 4. 训练模型以及评估
对如上三种不同的向量表示法，分别训练逻辑回归模型，需要做：
- 搭建模型
- 训练模型（并做交叉验证）
- 输出最好的结果

#### 4.1 用word2vec训练模型

In [39]:
from sklearn.linear_model import LogisticRegression

In [54]:
# 利用GriSearcCV找到最优参数组合
from sklearn.model_selection import GridSearchCV #网格搜索选出最好的参数组合
parameters = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],'penalty':['l1','l2'], 'solver':['liblinear','lbfgs','sag','saga'] }
LR = LogisticRegression()
LR_grid_search = GridSearchCV(estimator = LR,
                             param_grid = parameters,
                             cv = 5,
                             n_jobs = -1)
LR_grid_search.fit(word2vec_train,y_train)

print('Best Score:{} %'.format(LR_grid_search.best_score_))
print('Best Parameters:{} %'.format(LR_grid_search.best_params_))

Best Score:0.8875 %
Best Parameters:{'C': 10, 'penalty': 'l1', 'solver': 'saga'} %


D:\Downloads\Anaconda\envs\NLP_class\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [74]:
# 用非最优参数测试
LR_best=LogisticRegression(penalty='l1',C=100,solver='liblinear')
LR_best.fit(word2vec_train,y_train)
word2vec_y_pred=LR_best.predict(word2vec_test)
print('word2vec LR test accuracy %s' % metrics.accuracy_score(y_test,word2vec_y_pred))
print('Word2vec LR test F1_score %s' % metrics.f1_score(y_test, word2vec_y_pred,average="macro"))

word2vec LR test accuracy 0.9
Word2vec LR test F1_score 0.7222222222222222


In [59]:
# 用最优参数测试，发现效果居然没上面好
LR_best=LogisticRegression(C= 10, penalty='l1', solver= 'saga')
LR_best.fit(word2vec_train,y_train)
word2vec_y_pred=LR_best.predict(word2vec_test)
print('word2vec LR test accuracy %s' % metrics.accuracy_score(y_test,word2vec_y_pred))
print('Word2vec LR test F1_score %s' % metrics.f1_score(y_test, word2vec_y_pred,average="macro"))

word2vec LR test accuracy 0.8
Word2vec LR test F1_score 0.4444444444444444


D:\Downloads\Anaconda\envs\NLP_class\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


#### 4.2 用bert训练模型

In [101]:
bert_train[np.isnan(bert_train)] = 0 #将空值设为0

In [103]:
lr = LogisticRegression()

parameters = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],'penalty':['l1','l2'], 'solver':['liblinear','lbfgs','sag','saga'] }

lr_grid_search = GridSearchCV(estimator = lr,
                                   param_grid = parameters,
                                   cv = 5,
                                    n_jobs = -1)

lr_gs = lr_grid_search.fit(bert_train, y_train)
lr_best_score = lr_grid_search.best_score_
lr_bes_param = lr_grid_search.best_params_

print('Best Score:{}'.format(lr_best_score))
print('Best Parameters:{}'.format(lr_bes_param))

Best Score:0.825
Best Parameters:{'C': 0.0001, 'penalty': 'l1', 'solver': 'saga'}


In [104]:
bert_test[np.isnan(bert_test)] = 0

In [105]:
lr_best = LogisticRegression(penalty='l1',C=0.0001,solver='saga')
lr_best.fit(bert_train, y_train)
bert_y_pred = lr_best.predict(bert_test)

print('Bert LR test accuracy %s' % metrics.accuracy_score(y_test, bert_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('Bert LR test F1_score %s' % metrics.f1_score(y_test, bert_y_pred,average="macro"))

Bert LR test accuracy 0.85
Bert LR test F1_score 0.45945945945945943


#### 保存和加载模型

In [108]:
#  将bert训练出的模型存储下来，sklearn中的模型一般都用pickle.dump(model,f)来保存
import pickle
with open('bert_lr_star_predict.model','wb') as f:
    pickle.dump(lr_best,f)

In [109]:
# 加载模型方法：
with open('bert_lr_star_predict.model', 'rb') as f:
    lr_best = pickle.load(f)
    # 照常引用模型方法：
    print(lr_best.predict(bert_test))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


### 使用bert，并结合朴素贝叶斯模型

In [106]:
from sklearn.naive_bayes import GaussianNB
#使用bert特征训练一个朴素贝叶斯模型
gnb_bert = GaussianNB().fit(bert_train,y_train)
#使用朴素贝叶斯模型得到的测试集上的预测值
test_pred_gnb_bert = gnb_bert.predict(bert_test)
#朴素贝叶斯模型在测试集上准确率
print('Bert NaiveBayes test accuracy %s' % metrics.accuracy_score(y_test, test_pred_gnb_bert))
#朴素贝叶斯模型在测试集上的F1_Score
print('Bert NaiveBayes test F1_score %s' % metrics.f1_score(y_test, test_pred_gnb_bert,average="macro"))

Bert NaiveBayes test accuracy 0.8
Bert NaiveBayes test F1_score 0.6875


### 5. 实现打分

In [78]:
# 输入评论文本
queries=["复仇者联盟","差评差评"]
queries_list=[]
for query in queries:
    words=comment_cut(query)
    pure_query=rm_stop_word(words)
    query_vec=get_sentence_vec(pure_query)
    queries_list.append(query_vec)
    
query_predict=LR_best.predict(np.array(queries_list))
labels=[]
for i in query_predict:
    if i==1:
        labels.append("好电影")
    elif i==0:
        labels.append("烂电影")
for movie in range(len(queries)):
    print(queries[movie],":",labels[movie])

复仇者联盟 : 好电影
差评差评 : 烂电影
